# Corriendo Varias Trayectorias HYSPLIT en ARGOS para diferentes tiempos

Se presentan los pasos para ejecutar HYSPLIT empleando la interfase de Jupyter para trayectorias.

1. Crear y ubicarse en el directorio de trabajo
1. Crear el archivo ASCDATA.CFG 
1. Configurar los parametros de la ejecución de trayectoria
1. Crear el archivo CONTROL
1. Ejecutar el programa de trayectorias
1. Crear el despliegue

In [10]:
%%bash -l
# en la variable DIR definir el nombre del directorio
DIR=enero
# se revisa si existe en el directorio $HOME si no se crea

if [ ! -d ${HOME}/data/$DIR ];then mkdir $HOME/data/$DIR ;fi

# se ubica en el directorio
cd ${HOME}/data/${DIR}
#-------------------------------------------------------------
# set default directory structure if not passed through

  MDL="${HOME}/data/hysplit.v5.2.0_CentOS7.9.2009"
  OUT="${HOME}/data/${DIR}"
  MET="${HOME}/data/hysplit_data"
  cd $OUT

  if [ ! -f ASCDATA.CFG ]; then
     echo "-90.0  -180.0"     >ASCDATA.CFG 
     echo "0.1     0.1"      >>ASCDATA.CFG
     echo "1800   3600"      >>ASCDATA.CFG
     echo "2"                >>ASCDATA.CFG
     echo "0.2"              >>ASCDATA.CFG
     echo "'$MDL/bdyfiles/bdyfiles0p1/'" >>ASCDATA.CFG
  fi
  echo "### para Enero ###"


#----------------------------------------------------------
# Configura archivo de control para  simulaciones de 
#  dispersion/concentracion 

iyr=19  # Anio solo los dos ultimos dígitos
ims=01  # Mes dos digitos
idy=01  # Dia de inicio de modelacion
ihr=06  # Hora de inicio de modelacion

#  Ubicación de la fuente a modelar y altura
lat=19.7208   
lon=-96.406
lvl=100.0
#  
vop=0         # Opcion de movimiento vertical 
run=12        # Numero de horas a modelar
ztop=10000.0   # Altura del tope de modelacion

nmf=1         # Numero de archivos meteorologicos 
MET="${HOME}/hysplit_data" # Directorio donde estan los data meteorologicos


for idy in 01 06 15 24
do
data="AR_${ims}${idy}20${iyr}.bin"  #nombre del archivo meteorologico

#--------------------------------------------------------------
#   Indica cada cuantas horas va a correr
stph=01
#   Indica el numero de dias a correr
idia=00
#   Indica el numero de horas despues de iniciar.
hfin=30

#--------------------------------------------------------------
#  calcula las horas totales
if [ $idia -lt 1 ] ; then
   ht=$(($hfin+10#${ihr}))
   else
    ht=$((10#${ihr}+${idia}*24+10#${hfin}))
   fi
#
# dias en el mes
#    D   E  F  M  A  M  J  J  A  S  O  N  D          
mes=(31 31 28 31 30 31 30 31 31 30 31 30 31)

#--------------------------------------------------------------
# Datos de tiempo en loop

for ih in $( seq $ihr $stph $ht )
do
 if [ $ih -gt 23 ] ; then
   lh=$(($ih -24*$(($ih/24))))
   ld=$((${idy}+ $ih/24 ))
   if [ $ld -lt 10 ]; then ld=0$ld ;fi
  else
   lh=$ih
   ld=$idy
   lm=$ims
 fi
 if [ $ld -gt ${mes[$ims]} ]; then
   lm=$(($ims + 1))
   ld=$((${idy}+ $ih/24 - ${mes[$ims]}))
   if [ $lm -lt 10 ]; then lm=0$lm ;fi
   if [ $ld -lt 10 ]; then ld=0$ld ;fi
 fi
if [ $lh -lt 10 ]; then lh=0$lh ;fi


# Creacion del archivo CONTROL
#----------------------------------------------------------
# 

  echo "$iyr $lm $ld $lh       " >CONTROL
  echo "1                      ">>CONTROL
  echo "$lat $lon $lvl         ">>CONTROL
  echo "$run                   ">>CONTROL
  echo "$vop                   ">>CONTROL
  echo "$ztop                  ">>CONTROL
  echo "$nmf                   ">>CONTROL
  echo "$MET/                  ">>CONTROL
  echo "$data                  ">>CONTROL
  echo "$OUT/                  ">>CONTROL
  echo "tdump$iyr$lm$ld$lh     ">>CONTROL
 
#----------------------------------------------------------
# corre la simulacion

  rm -f SETUP.CFG
    
  ${MDL}/exec/hyts_std >sal
    echo "Terminado $iyr$lm$ld$lh"
done 
done # para cada día

### para Enero ###
Terminado 19010106
Terminado 19010107
Terminado 19010108
Terminado 19010109
Terminado 19010110
Terminado 19010111
Terminado 19010112
Terminado 19010113
Terminado 19010114
Terminado 19010115
Terminado 19010116
Terminado 19010117
Terminado 19010118
Terminado 19010119
Terminado 19010120
Terminado 19010121
Terminado 19010122
Terminado 19010123
Terminado 19010200
Terminado 19010201
Terminado 19010202
Terminado 19010203
Terminado 19010204
Terminado 19010205
Terminado 19010206
Terminado 19010207
Terminado 19010208
Terminado 19010209
Terminado 19010210
Terminado 19010211
Terminado 19010212
Terminado 19010606
Terminado 19010607
Terminado 19010608
Terminado 19010609
Terminado 19010610
Terminado 19010611
Terminado 19010612
Terminado 19010613
Terminado 19010614
Terminado 19010615
Terminado 19010616
Terminado 19010617
Terminado 19010618
Terminado 19010619
Terminado 19010620
Terminado 19010621
Terminado 19010622
Terminado 19010623
Terminado 19010700
Terminado 19010701
Terminado 19

In [12]:
%%bash -l

ls  ../data/enero/tdump* |wc -l
tail -1 ../data/enero/tdump19012512
#cat ../data/enero/tdump19012512

124
     1     1    19     1    25    23     0    47    11.0   18.788  -96.493   1340.6    860.1


## Visualización de las trayectorias


In [13]:
%%bash -l
# en la variable DIR definir el nombre del directorio
DIR=enero
MDL="${HOME}/data/hysplit.v5.2.0_CentOS7.9.2009"


# se revisa si existe en el directorio $HOME si no se crea

if [ ! -d ${HOME}/data/$DIR ];then mkdir $HOME/data/$DIR ;fi

# se ubica en el directorio
cd ${HOME}/data/${DIR}
# Genera el archivo de todas las trayectorias
#ls -r tdump1901011* > INFILE
ls tdump1901*  > INFILE

 WEB=""
  if [ -f /bin/convert ];then WEB="/bin/convert";fi

echo "'$MDL/graphics/mexico.shp' 0 0.01 0.4 0.6 0.8" >shapefiles_mex.txt

#-------------------------------------------------------------
# set default directory structure if not passed through


echo "'TITLE&','Trayectorias de Enero 01 06 &'"  >LABELS.CFG
${MDL}/exec/trajplot  +g0 -i+INFILE -jshapefiles_mex.txt -l3 -v4 -otray.ps -z0 -h19.7:-96.4 

${WEB} tray.ps tray.jpg
  

 Started Trajectory Drawing
 HYSPLIT version: hysplit.v5.2.0
 Last Changed Date: 2022-01-16
 Processing: tdump19010106                           
 Processing: tdump19010107                           
 Processing: tdump19010108                           
 Processing: tdump19010109                           
 Processing: tdump19010110                           
 Processing: tdump19010111                           
 Processing: tdump19010112                           
 Processing: tdump19010113                           
 Processing: tdump19010114                           
 Processing: tdump19010115                           
 Processing: tdump19010116                           
 Processing: tdump19010117                           
 Processing: tdump19010118                           
 Processing: tdump19010119                           
 Processing: tdump19010120                           
 Processing: tdump19010121                           
 Processing: tdump19010122                  